In [2]:
import openfermion as of
from sdstate import *
import pickle
import numpy as np
path = "./Hamiltonians/"
mol = "LiH"
with open(path + mol + ".pkl", "rb") as f:
    Hf = pickle.load(f)
spin_orbs = of.count_qubits(Hf)
print(f"Number of spin orbitals: {spin_orbs}")

def get_1e_2e_tensor(Hf, n):
#     Get the tensor from FermionOperator, in spin orbital basis, as a tuple of 1e and 2e tensor
    Hf = of.chemist_ordered(Hf)
    H1e = np.zeros((n,n))
    H2e = np.zeros((n,n,n,n))
    for term, coeff in Hf.terms.items():
        if len(term) == 2:
            H1e[term[0][0], term[1][0]] = coeff
        elif len(term) ==4:
            H2e[term[0][0], term[1][0],term[2][0], term[3][0]] = coeff
    return H1e, H2e
H1e, H2e = get_1e_2e_tensor(Hf, spin_orbs)
# print(Hf)
# Remove constant term to eliminate constant shif
Hf -= Hf.terms[()]
neutral_enum = 4
# The range of electron number subspaces to search
delta_enum = 2
e_nums = list(range(neutral_enum - delta_enum, neutral_enum + delta_enum))
print(lanczos_total_range(Hf, 2, e_nums = e_nums))
print(lanczos_total_range((H1e, H2e), 2, e_nums = e_nums))
state1 = sdstate(int("111100000000", 2), n_qubit = spin_orbs)
state2 = sdstate(int("000000001111", 2), n_qubit = spin_orbs)
states = [state1, state2]
print(lanczos_total_range(Hf, 2, states = states))

Number of spin orbitals: 12
(-1.865888555575717, -13.15094535007822)
(-1.8658885555757156, -13.150945350078224)
(-2.8243630906151043, -13.172472760779392)
